In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'Goiás - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [17]:
data = pd.read_csv('2003_01_model_input_GO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),Goiais-03anhão - Valor Cimento Portland,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Consumo de Cimento (t)
0,2003,8.179785,0.745293,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,707.997208,3.006392,9.371822e+07,5.518325e+06,14.031270,8.418248e+07,1311.935000
1,2004,8.111394,0.746636,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,747.779683,2.749852,9.513036e+07,5.577901e+06,14.069740,8.464257e+07,1514.939000
2,2005,8.088033,0.747851,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,773.277269,2.602174,9.643551e+07,5.616938e+06,14.103822,8.501827e+07,1716.900000
3,2006,8.002287,0.748941,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,813.650774,2.723996,9.771534e+07,5.664472e+06,14.152999,8.544612e+07,1730.142000
4,2007,7.971612,0.749908,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,924.314058,3.192897,9.885878e+07,5.710316e+06,14.189755,8.577093e+07,1943.769000
5,2008,7.941146,0.750753,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,1049.218132,3.597718,9.979485e+07,5.727076e+06,14.203138,8.592012e+07,2310.616000
6,2009,7.969157,0.751479,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,1167.838500,3.778583,1.005138e+08,5.757645e+06,14.197250,8.596210e+07,2335.717000
7,2010,7.932921,0.752088,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,1306.390438,3.922305,1.030813e+08,5.806966e+06,14.241313,8.663480e+07,2657.527000
8,2011,7.921619,0.752581,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,1430.447037,4.039003,1.054535e+08,5.798734e+06,14.278825,8.718791e+07,2901.707000
9,2012,7.786290,0.752723,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,1500.429389,4.131063,1.075784e+08,5.775231e+06,14.289292,8.762067e+07,3061.533000


In [18]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),Goiais-03anhão - Valor Cimento Portland,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
0,8.179785,0.745293,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,707.997208,3.006392,9.371822e+07,5.518325e+06,14.031270,8.418248e+07
1,8.111394,0.746636,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,747.779683,2.749852,9.513036e+07,5.577901e+06,14.069740,8.464257e+07
2,8.088033,0.747851,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,773.277269,2.602174,9.643551e+07,5.616938e+06,14.103822,8.501827e+07
3,8.002287,0.748941,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,813.650774,2.723996,9.771534e+07,5.664472e+06,14.152999,8.544612e+07
4,7.971612,0.749908,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,924.314058,3.192897,9.885878e+07,5.710316e+06,14.189755,8.577093e+07
5,7.941146,0.750753,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,1049.218132,3.597718,9.979485e+07,5.727076e+06,14.203138,8.592012e+07
6,7.969157,0.751479,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,1167.838500,3.778583,1.005138e+08,5.757645e+06,14.197250,8.596210e+07
7,7.932921,0.752088,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,1306.390438,3.922305,1.030813e+08,5.806966e+06,14.241313,8.663480e+07
8,7.921619,0.752581,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,1430.447037,4.039003,1.054535e+08,5.798734e+06,14.278825,8.718791e+07
9,7.786290,0.752723,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,1500.429389,4.131063,1.075784e+08,5.775231e+06,14.289292,8.762067e+07


In [19]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1514.939000
1     1716.900000
2     1730.142000
3     1943.769000
4     2310.616000
5     2335.717000
6     2657.527000
7     2901.707000
8     3061.533000
9     3181.778000
10    2861.897429
11    2517.290743
12    2193.545829
13    2270.346000
14    2282.903000
15    2367.716000
16    2438.585000
17    2782.135000
18    2807.973000
19            NaN
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [20]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),Goiais-03anhão - Valor Cimento Portland,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
0,-0.419401,-2.134050,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.556914,-1.176939,-1.653672,-1.577371,-2.271887,-1.973835
1,-0.545138,-1.684552,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.444254,-1.552293,-1.438444,-0.939271,-1.704457,-1.604809
2,-0.588088,-1.277751,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.372048,-1.768367,-1.239523,-0.521162,-1.201734,-1.303461
3,-0.745733,-0.912947,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.257714,-1.590125,-1.044463,-0.012043,-0.476356,-0.960295
4,-0.802129,-0.589437,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.944328,-0.904056,-0.870187,0.478975,0.065791,-0.699769
5,-0.858140,-0.306523,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.590614,-0.311745,-0.727519,0.658485,0.263200,-0.580109
6,-0.806643,-0.063503,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.254694,-0.047115,-0.617937,0.985892,0.176354,-0.546440
7,-0.873262,0.140325,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.137669,0.163171,-0.226624,1.514151,0.826284,-0.006881
8,-0.894041,0.305660,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.488984,0.333916,0.134936,1.425985,1.379601,0.436759
9,-1.142844,0.353208,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.687166,0.468614,0.458795,1.174255,1.533993,0.783868


In [21]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1514.939000
1     1716.900000
2     1730.142000
3     1943.769000
4     2310.616000
5     2335.717000
6     2657.527000
7     2901.707000
8     3061.533000
9     3181.778000
10    2861.897429
11    2517.290743
12    2193.545829
13    2270.346000
14    2282.903000
15    2367.716000
16    2438.585000
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [22]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),Goiais-03anhão - Valor Cimento Portland,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
17,0.596674,-0.104074,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.913043,1.914658,0.832746,-0.850806,-0.449768,0.60687


In [23]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    2782.135
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [24]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [27]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [28]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3317632302, 171150871, 3066446328, 1079218211, 3722090730, 1472646180, 2186790848, 2307420015, 2300711256, 3899746283, 956361001, 1263235055, 765992751, 3709733017, 2810233357, 537738576, 4013269454, 3791757038, 3980333594, 3394048614, 3091817628, 3412426208, 3875477579, 1701074920, 2708931009, 227706077, 1925934537, 916142755, 383433419, 1956390367, 225717191, 3881700292, 3117476708, 264364044, 137754078, 4159689104, 2754175603, 1655487076, 1282242365, 2736204350, 295880981, 2891055549, 1316890156, 3397871693, 576959103, 691097876, 1590472083, 2137294500, 984105692, 1373087523, 328362593, 2465175505, 2061843017, 3431651918, 200930587, 1976252219, 1676358608, 1466442058, 2610660427, 2438320287, 1243113616, 2404848863, 751650245, 867448144, 950795822, 1461177438, 684719440, 2487616851, 4271868501, 2041591500, 1307023718, 1464736156, 973558208, 1494855840, 1413329488, 2549266067, 2818025806, 1086301873, 3017129108, 2723826396, 1620613973, 2856455391, 1453884097, 2836665403, 742478828, 2

2023-10-01 12:52:21.627836: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 4068.861328125


Step: 40 ___________________________________________
val_loss: 4050.375


Step: 41 ___________________________________________
val_loss: 4050.463623046875


Step: 42 ___________________________________________
val_loss: 842.246826171875
winner_seed: 1316890156


Step: 43 ___________________________________________
val_loss: 2123.384521484375


Step: 44 ___________________________________________
val_loss: 3307.907958984375


Step: 45 ___________________________________________


2023-10-01 12:53:59.388085: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 4050.27734375


Step: 46 ___________________________________________
val_loss: 4050.279541015625


Step: 47 ___________________________________________
val_loss: 2049.08740234375


Step: 48 ___________________________________________
val_loss: 2661.83642578125


Step: 49 ___________________________________________
val_loss: 6821.44921875


Step: 50 ___________________________________________
val_loss: 4050.27734375


Step: 51 ___________________________________________
val_loss: 1248.965576171875


Step: 52 ___________________________________________
val_loss: 4054.621826171875


Step: 53 ___________________________________________
val_loss: 4050.397216796875


Step: 54 ___________________________________________
val_loss: 4050.280029296875


Step: 55 ___________________________________________
val_loss: 17303.5546875


Step: 56 ___________________________________________
val_loss: 13252.2578125


Step: 57 ___________________________________________
val_loss: 4050.28295898437

2023-10-01 13:02:26.335645: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-01 13:02:26.368806: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 1357.3753662109375


Step: 76 ___________________________________________
val_loss: 2486.572265625


Step: 77 ___________________________________________
val_loss: 4050.27734375


Step: 78 ___________________________________________


2023-10-01 13:03:29.160133: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-01 13:03:29.195954: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 4050.279052734375


Step: 79 ___________________________________________
val_loss: 4050.280029296875


Step: 80 ___________________________________________
val_loss: 4050.715576171875


Step: 81 ___________________________________________
val_loss: 4050.614990234375


Step: 82 ___________________________________________
val_loss: 1205.41552734375


Step: 83 ___________________________________________
val_loss: 4056.196533203125


Step: 84 ___________________________________________
val_loss: 4050.29296875


Step: 85 ___________________________________________
val_loss: 2239.3076171875


Step: 86 ___________________________________________
val_loss: 2130.856689453125


Step: 87 ___________________________________________
val_loss: 4050.574951171875


Step: 88 ___________________________________________
val_loss: 4050.287841796875


Step: 89 ___________________________________________
val_loss: 4050.277099609375


Step: 90 ___________________________________________
val_loss: 4

In [29]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 1s 853ms/step - loss: 5825959.5000 - val_loss: 5366466.5000
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 5607700.0000 - val_loss: 13496903.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 8595847.0000 - val_loss: 5763682.5000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 5463029.5000 - val_loss: 10095733.0000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 5257366.5000 - val_loss: 7860993.5000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 5102274.5000 - val_loss: 5925509.5000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 4481300.0000 - val_loss: 6792540.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 4779144.0000 - val_loss: 8888145.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 5215169.5000 - val_loss: 63

Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 976795.3125 - val_loss: 229600.7031
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 733773.9375 - val_loss: 266532.7500
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 960818.4375 - val_loss: 270787.5312
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 723775.2500 - val_loss: 321610.4688
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 755033.7500 - val_loss: 308613.7500
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 708342.0625 - val_loss: 292049.1250
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 759477.5000 - val_loss: 515673.4688
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 719827.8125 - val_loss: 397352.2500
Epoch 82/10000
1/1 [==============================] - 0s 16ms/step - loss: 668277.3125 - val_loss: 214579.6250
E

1/1 [==============================] - 0s 14ms/step - loss: 328900.6562 - val_loss: 174935.7344
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 400864.2812 - val_loss: 195886.0000
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 265904.6562 - val_loss: 206056.1250
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 282414.9062 - val_loss: 208022.7031
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 338352.2500 - val_loss: 204596.5156
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 244248.7188 - val_loss: 207173.2500
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 444672.1562 - val_loss: 161363.8906
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 321721.2812 - val_loss: 233433.2969
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 263051.7188 - val_loss: 166790.8750
Epoch 15

1/1 [==============================] - 0s 14ms/step - loss: 186807.1719 - val_loss: 34412.8555
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 164625.0000 - val_loss: 32437.2090
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 210560.2969 - val_loss: 48319.0938
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 133020.3906 - val_loss: 57311.9492
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 208679.2656 - val_loss: 29336.1406
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 144266.5625 - val_loss: 33128.9062
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 144891.2188 - val_loss: 66517.1953
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 148534.2656 - val_loss: 60954.1055
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 142392.0781 - val_loss: 57485.8750
Epoch 229/10000
1

1/1 [==============================] - 0s 14ms/step - loss: 128835.5703 - val_loss: 32977.4727
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 130667.6641 - val_loss: 37416.1523
Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 129323.0859 - val_loss: 33140.8086
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 154478.4844 - val_loss: 29165.3887
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 155969.4219 - val_loss: 19102.0625
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 112099.5625 - val_loss: 48642.7383
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 101315.2891 - val_loss: 37868.3086
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 86022.6797 - val_loss: 39927.2344
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 120110.9531 - val_loss: 21777.3906
Epoch 303/10000
1/

...

1/1 [==============================] - 0s 14ms/step - loss: 25644.7715 - val_loss: 2926.7588
Epoch 1193/10000
1/1 [==============================] - 0s 14ms/step - loss: 26124.9883 - val_loss: 2803.5413
Epoch 1194/10000
1/1 [==============================] - 0s 14ms/step - loss: 30520.3457 - val_loss: 6023.8394
Epoch 1195/10000
1/1 [==============================] - 0s 14ms/step - loss: 21837.3320 - val_loss: 1266.2708
Epoch 1196/10000
1/1 [==============================] - 0s 14ms/step - loss: 35263.1719 - val_loss: 1154.1288
Epoch 1197/10000
1/1 [==============================] - 0s 14ms/step - loss: 14306.5703 - val_loss: 1073.3817
Epoch 1198/10000
1/1 [==============================] - 0s 14ms/step - loss: 25682.8066 - val_loss: 3318.0977
Epoch 1199/10000
1/1 [==============================] - 0s 14ms/step - loss: 25849.8105 - val_loss: 4360.8325
Epoch 1200/10000
1/1 [==============================] - 0s 14ms/step - loss: 19408.3633 - val_loss: 5058.2710
Epoch 1201/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 24750.9707 - val_loss: 1910.5643
Epoch 1267/10000
1/1 [==============================] - 0s 14ms/step - loss: 28941.3242 - val_loss: 3548.2307
Epoch 1268/10000
1/1 [==============================] - 0s 14ms/step - loss: 31912.8340 - val_loss: 5598.7407
Epoch 1269/10000
1/1 [==============================] - 0s 14ms/step - loss: 25463.3711 - val_loss: 1383.1187
Epoch 1270/10000
1/1 [==============================] - 0s 13ms/step - loss: 21453.5625 - val_loss: 1198.9994
Epoch 1271/10000
1/1 [==============================] - 0s 14ms/step - loss: 28831.8398 - val_loss: 1245.6788
Epoch 1272/10000
1/1 [==============================] - 0s 14ms/step - loss: 28599.0527 - val_loss: 3616.3125
Epoch 1273/10000
1/1 [==============================] - 0s 14ms/step - loss: 29977.1465 - val_loss: 2587.1643
Epoch 1274/10000
1/1 [==============================] - 0s 14ms/step - loss: 25301.7656 - val_loss: 2864.6492
Epoch 1275/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 28900.0586 - val_loss: 4351.6094
Epoch 1341/10000
1/1 [==============================] - 0s 14ms/step - loss: 55529.5547 - val_loss: 8420.7920
Epoch 1342/10000
1/1 [==============================] - 0s 14ms/step - loss: 25684.3281 - val_loss: 4444.4277
Epoch 1343/10000
1/1 [==============================] - 0s 14ms/step - loss: 27323.1660 - val_loss: 4465.0142
Epoch 1344/10000
1/1 [==============================] - 0s 14ms/step - loss: 23760.6758 - val_loss: 4452.7031
Epoch 1345/10000
1/1 [==============================] - 0s 14ms/step - loss: 23716.5332 - val_loss: 4527.4331
Epoch 1346/10000
1/1 [==============================] - 0s 14ms/step - loss: 24548.5215 - val_loss: 4546.9292
Epoch 1347/10000
1/1 [==============================] - 0s 14ms/step - loss: 25965.3164 - val_loss: 4785.3218
Epoch 1348/10000
1/1 [==============================] - 0s 14ms/step - loss: 28460.0625 - val_loss: 4578.0933
Epoch 1349/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 27792.6348 - val_loss: 2674.5012
Epoch 1415/10000
1/1 [==============================] - 0s 14ms/step - loss: 23070.0938 - val_loss: 2401.0078
Epoch 1416/10000
1/1 [==============================] - 0s 14ms/step - loss: 24890.2969 - val_loss: 2482.4438
Epoch 1417/10000
1/1 [==============================] - 0s 14ms/step - loss: 26603.8066 - val_loss: 1727.9917
Epoch 1418/10000
1/1 [==============================] - 0s 14ms/step - loss: 25335.3164 - val_loss: 4011.2637
Epoch 1419/10000
1/1 [==============================] - 0s 14ms/step - loss: 24594.0488 - val_loss: 2165.1082
Epoch 1420/10000
1/1 [==============================] - 0s 14ms/step - loss: 24752.6387 - val_loss: 3307.9172
Epoch 1421/10000
1/1 [==============================] - 0s 14ms/step - loss: 18819.3535 - val_loss: 1120.0730
Epoch 1422/10000
1/1 [==============================] - 0s 14ms/step - loss: 20559.9844 - val_loss: 2424.3462
Epoch 1423/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 21150.0020 - val_loss: 12738.8398
Epoch 1489/10000
1/1 [==============================] - 0s 14ms/step - loss: 18073.6133 - val_loss: 13044.1182
Epoch 1490/10000
1/1 [==============================] - 0s 14ms/step - loss: 17623.7656 - val_loss: 12420.4033
Epoch 1491/10000
1/1 [==============================] - 0s 14ms/step - loss: 16646.8164 - val_loss: 13003.9736
Epoch 1492/10000
1/1 [==============================] - 0s 14ms/step - loss: 16858.2285 - val_loss: 12323.0029
Epoch 1493/10000
1/1 [==============================] - 0s 14ms/step - loss: 23129.7441 - val_loss: 12080.9404
Epoch 1494/10000
1/1 [==============================] - 0s 14ms/step - loss: 16483.7773 - val_loss: 12918.0186
Epoch 1495/10000
1/1 [==============================] - 0s 14ms/step - loss: 17230.6816 - val_loss: 12004.7305
Epoch 1496/10000
1/1 [==============================] - 0s 14ms/step - loss: 26576.0664 - val_loss: 13111.3564
Epoch 1497/10000
1

In [30]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 50ms/step
[2782.135](test_target) - [[2515.2134]](prediction) = 266.9216210937502


In [31]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [32]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),Goiais-03anhão - Valor Cimento Portland,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
0,1.832928,-1.708338,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.119467,-0.397040,-1.556881,-1.701956,-1.711402,-1.698591
1,1.025450,-1.108653,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-0.924403,-0.932149,-1.061470,-1.044093,-1.150571,-1.080656
2,0.749626,-0.565930,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.799382,-1.240186,-0.603594,-0.613035,-0.653693,-0.576048
3,-0.262764,-0.079236,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.601421,-0.986082,-0.154604,-0.088148,0.063250,-0.001415
4,-0.624935,0.352365,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.058811,-0.008016,0.246543,0.418077,0.599093,0.434837
5,-0.984636,0.729808,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.553626,0.836389,0.574937,0.603147,0.794207,0.635208
6,-0.653921,1.054027,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.135251,1.213649,0.827173,0.940694,0.708371,0.691586
7,-1.081748,1.325957,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.814607,1.513435,1.727896,1.485313,1.350743,1.595079


train_target:


,Goiás - Consumo de Cimento (t)
0,1514.939
1,1716.900
2,1730.142
3,1943.769
4,2310.616
5,2335.717
6,2657.527
7,2901.707


test_input:


...

,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),Goiais-03anhão - Valor Cimento Portland,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
17,0.596674,-0.104074,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.913043,1.914658,0.832746,-0.850806,-0.449768,0.60687


test_target:


,Goiás - Consumo de Cimento (t)
17,2782.135


1/1 [==============================] - 0s 34ms/step
Error: 266.9216210937502




[2590.6005859375,
 2753.36962890625,
 2956.861572265625,
 2958.89306640625,
 2543.6474609375,
 2753.41943359375,
 2398.756103515625,
 2244.200439453125,
 2371.176025390625,
 2515.21337890625]

In [33]:
display(targets)
display(predictions)

[3061.533,
 3181.778,
 2861.8974285714285,
 2517.2907428571425,
 2193.5458285714285,
 2270.346,
 2282.903,
 2367.716,
 2438.585,
 2782.135]

[2590.6005859375,
 2753.36962890625,
 2956.861572265625,
 2958.89306640625,
 2543.6474609375,
 2753.41943359375,
 2398.756103515625,
 2244.200439453125,
 2371.176025390625,
 2515.21337890625]

In [34]:
mae = mean_absolute_error(predictions, targets)
mae

284.27815781250007

In [35]:
porcentage = mae/np.mean(targets)
porcentage

0.10951580042341919